Test the data queuing and distortion and augmentation

In [ ]:
import numpy as np
import tensorflow as tf
from local_lr import cifar10_input
import os
import tarfile
import sys
from six.moves import urllib

import matplotlib.pyplot as plt
% matplotlib inline

In [ ]:
# Load data from Alex websited and save locally

FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string("data_dir", "/tmp/cifar10_data", """Path to the CIFAR10-data director""")
tf.app.flags.DEFINE_integer('batch_size', 128, """Numbers of images to process in a batch.""")


DATA_URL = 'https://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz'

if not FLAGS.data_dir:
    raise ValueError('Please supply a data_dir')
    
data_dir = os.path.join(FLAGS.data_dir, 'cifar10-batches-bin')

dest_directory = FLAGS.data_dir

if not os.path.exists(dest_directory):
    os.makedirs(dest_directory)
    
filename = DATA_URL.split('/')[-1]
filepath = os.path.join(dest_directory, filename)

if not os.path.exists(filepath):
    filepath, _ = urllib.request.urlretrieve(DATA_URL, filepath)
    statsinfo = os.stat(filepath)
    print('Successfullly downloaded', filename, statsinfo.st_size, '.bytes')
extracted_dir_path = os.path.join(dest_directory, 'cifar-10-batches-bin')

if not os.path.exists(extracted_dir_path):
    tarfile.open(filepath, 'r:gz').extractall(dest_directory)

Successfullly downloaded cifar-10-binary.tar.gz 170052171 .bytes


In [ ]:
# Create shuffled or unshuffled batch
def _generate_image_and_label_batch(image, label, min_queue_examples, batch_size, shuffle):
    
    num_process_thread = 16
    if shuffle:
        images, label_batch = tf.train.shuffle_batch([image, label], batch_size=batch_size, num_threads=num_process_thread, 
                                                    capacity=min_queue_examples + 3 * batch_size, min_after_dequeue=min_queue_examples)
    else:
        images, label_batch = tf.train.batch([image, label], batch_size=batch_size, num_threads=num_process_thread, capacity=min_queue_examples + 3 * batch_size)
    
    tf.summary.image('images', images)
    
    return images, tf.reshape(label_batch, [batch_size])

In [ ]:
# Extract image data from the queue information and return object with image, label and other information

def read_cifar10(filename_queue):
    
    class CIFAR10Record(object):
        pass
    
    result = CIFAR10Record()
    
    label_bytes = 1
    result.height = 32
    result.width = 32
    result.depth = 3
    
    image_bytes = result.height * result.width + result.depth
    
    record_bytes = label_bytes + image_bytes
    
    reader = tf.FixedLengthRecordReader(record_bytes=record_bytes)
    result.key, value = reader.read(filename_queue)
    
    record_bytes = tf.decode_raw(value, tf.uint8)
    result.label = tf.cast(tf.strided_slice(record_bytes, [0], [label_bytes]), tf.int32)
    
    depth_major = tf.reshape(tf.strided_slice(record_bytes, [label_bytes], [label_bytes+image_bytes]), [result.depth, result.height, result.width])
    result.unit8image = tf.transpose(depth_major, [1,2,0])
    
    return result

In [ ]:
# Generate images and labels after augmentation, pre-precessing and batching
data_dir = os.path.join(FLAGS.data_dir, 'cifar10-batches-bin')
# ages, labels = distorted_input(data_dir = data_dir, batch_size=FLAGS.batch_size)
IMAGE_SIZE = 32
NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = 50000

batch_size = FLAGS.batch_size

filename = [os.path.join(data_dir, 'data_batch_%d.bin' % i) for i in range(1, 6)]

filename_queue = tf.train.string_input_producer(filename)

with tf.name_scope('data_augmentation'):
    
    read_input = read_cifar10(filename_queue)
    reshaped_image = tf.cast(read_input.unit8image, tf.float32)
    
    height = IMAGE_SIZE
    width = IMAGE_SIZE
    
    distorted_image = tf.random_crop(reshaped_image, [height, width, 3])
    
    distorted_image = tf.image.random_flip_left_right(distorted_image)
    
    distorted_image = tf.image.random_brightness(distorted_image, max_delta=63)
    
    distorted_image = tf.image.random_contrast(distorted_image, lower=0.2, upper = 1.8)
    
    float_image = tf.image.per_image_standardization(distorted_image)
    
    float_image.set_shape([height, width, 3])
    read_input.label.set_shape([1])
    
    min_fraction_of_examples_in_queue = 0.4
    min_queue_examples = int(NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN * min_fraction_of_examples_in_queue)
    
    print('Filling queue with %d CIFAR images before starting to train.This will takge a few minutes.' % min_queue_examples)
curr_batch_image, curr_batch_label = _generate_image_and_label_batch(float_image, read_input.label, min_queue_examples, batch_size, shuffle=True)

Filling queue with 20000 CIFAR images before starting to train.This will takge a few minutes.


In [ ]:
# Build the network

In [ ]:
# Build the trainer

In [ ]:
# Evaluate model performance

In [ ]:
# Improve the model